### Building Pulse: A small autograd based neural network library

*This work is highly motivated by the awesome [video](https://youtu.be/VMj-3S1tku0) made by Andrej Karpathy.* 

In this project I will try to break down each piece of the puzzle of building a neural network and explain it in each step of the way. I would be as naive as possible and will always put understanding before performance.

I hope you will also enjoy this as much as I enjoyed building this from scratch.

First up is always importing the necessary libraries. Since, we are going to be doing very basic thing, I will just import math, and that's it. (I am not kidding :D )

In [1]:
## Importing necessary libraries ##


import math
import graphviz
import string
import random

The heart of our library is the **Pulse** module. It holds the basic operations. I intend to add more and more and I go on. 

So, let's get give your heart's pulse (pun!) a race and start building ourselves **Pulse**.

So, what are we going to do? What are the steps we need to take? Let's write these up in plain English, and I will setup checkboxes such that I can tick them as done once implemented.

- Creation of a class called Pulse. 

- We need to have a constructor of this class Pulse, which holds the data that is given by the user. Since, our Pulse only supports numerical value we must do a sanity check for the types. Our constructor must also hold a few other things: 
    - [x] grad : Gradient of the current pulse with respect to the last node. We initialize it as 0.0.
    - [x] children : The nodes from which the current Pulse node is created. It is initialized with an empty list.
    - [x] operation : The operation symbol that we are doing with the current node. It is a string. It is initialized with an empty string.
    - [x] name : Name of the node. Initialized with string.
    
- Adding operations. Each neuron in a neural network is manipulated via certain operations.

    - [x] First we implement the addition operation. This can be done via the python in-built method of ` __add__(self,other)`. This works when a number is added to any Pulse object. Eg. Pulse(4) + Pulse(5) works. We alse need to add the backward gradient of the nodes from which the resulting node is created. For addition, gradient is routed and hence, we just need to accumulate the gradient of the result node on the previous nodes. Apart from this, if we give an operation like Pulse(4) + 5 won't work since 5 is not a Pulse object. Hence if there is any
    - [x] The `__add__()` method doesn't work when there is 5 + Pulse(4) instead of Pulse(4) + 5. So we are needed to implement the `__radd__()` method and the same thing repeats here.
    - [x] Similar to the addition we add the multiplication operation. We do this via the `__mul__(self,other)` method of python classes. We also incorporate `__rmul__(self,other)` method to adjust for left multiplication. The backward pass of this neuron also calculates the gradient of the previous nodes. The gradient of previous nodes is very trivial and just takes into account the other nodes' value.
    - [x] The `__neg__` method does negation operation and the `__pow__` and `__rpow__` method does the power operation and are implemented accordingly.
    - [x] Similarly the implementation for division is done via `__truediv__` and `__rtruediv__` magic methods.
    - [x] We also implement the `tanh` method to incorporate tanh function.
    
- Adding `backward()` method, which topologically sorts all the nodes starting from the current calling node and then calculates the gradient of each of the previous nodes by first initiating the gradient of the current node to 1 and then calling the `._backward()` for each of the nodes one at a time. This calculates the gradient of each of the node with respect to the called node.

In [2]:
## Pulse module ##

class Pulse:
    """The core module of our neural network library."""
    def __init__(self, data, name = '' , children = [] , operation = ''):
        """Constructor."""
        
        assert type(data) == float or type(data) == int , "The data must be an int or a float!"
        
        self.data = data
        self.grad = 0.0
        self.name = name
        self._children = []
        self._children.extend(children)
        self._operation = operation
        self._backward = lambda : None
        
    
    def backward(self):
        """Does backward gradient calculation of current node with respect to
        all the previous nodes"""
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._children:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
    
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
        
        
    def __repr__(self):
        """String representation of Pulse."""
        
        return f"Pulse({self.data} , name = \"{self.name}\")"
    
    def __add__(self , other):
        """Addition operation of Pulse."""
        
        if not isinstance(other, Pulse):
            other = Pulse(other)
            
        result = Pulse(self.data + other.data, children = [self , other], operation='+')
        
        def _backward_():
            other.grad += 1.0 * result.grad
            self.grad += 1.0 * result.grad
        
        result._backward = _backward_ 
        
        return result
    
    def __radd__(self ,other):
        """Reverse Addition operation of Pulse."""
        
        return self + other
    
    def __mul__(self , other):
        """Multiplication operation of Pulse."""
        
        if not isinstance(other, Pulse):
            other = Pulse(other)
            
        result = Pulse(self.data * other.data, children = [self , other], operation='*')
        
        def _backward_():
            other.grad += self.data * result.grad
            self.grad += other.data * result.grad
        
        result._backward = _backward_ 
        
        return result
    
    def __rmul__(self ,other):
        """Reverse Multiplication operation of Pulse."""
        
        return self * other
    
    def __pow__(self , other):
        """Implements the power operation."""
        #print(self)
        other_grad = True
        
        if not isinstance(other, Pulse):
            other_grad = False
            other = Pulse(data = other, name = f'{other}')
        
        result = Pulse(self.data ** other.data , children = [self,other] , operation = '**')
        
        def _backward_():
            self.grad += (other.data) * ((self.data) ** (other.data - 1)) * (result.grad) 
            if other_grad : 
                other.grad += (self.data ** other.data) * math.log(self.data)
            
        result._backward = _backward_
        
        return result
    
    def __rpow__(self , other):
        """Reverse power."""
        other_grad = True
        if not isinstance(other , Pulse):
            other_grad = False
            other = Pulse(data = other , name = f'{other}')
            
        result = Pulse(other.data ** self.data , children = [self,other] , operation = '**')
        
        def _backward_():
            self.grad = (other.data ** self.data) * math.log(other.data)
            if other_grad:
                other.grad += (other.data) * ((self.data) ** (other.data - 1)) * (result.grad)
            
        result._backward = _backward_
        
        return result
    
    def __neg__(self):
        """Negation of Pulse."""
        
        return self * -1
    
    def __truediv__(self , other):
        """Division operation for pulse."""
        
        return self * (other ** -1)
    
    def __rtruediv__(self , other):
        """Division operation for pulse."""
        
        return other * self**-1
    
    def __sub__(self , other):
        """Substraction operation for Pulse."""
        return self + (-other)
    
    def tanh(self):
        """Tanh function."""
        if not isinstance(self , Pulse):
            self = Pulse(data = self , name = f'{self}')
        
        result = (math.exp(self.data) - math.exp(-self.data)) / (math.exp(self.data) + math.exp(-self.data)) 
        
        result = Pulse(result, children=[self], operation='tanh')
        
        def _backward_():
            self.grad += (1 - result.data ** 2) * result.grad
        
        result._backward = _backward_
        
        return result

We all love a Visualization toolkit. But, graph visualization can be a bit tricky. In our project we are going to use ```graphviz``` to visualize our graph. You can have a look at their [documentation](https://graphviz.readthedocs.io/en/stable/manual.html) to learn more.

As a next step we are going to implement our very own utility function for visualization. I am going to explain each step so we are on track to understanding it.

In [3]:
##The following code is entirely copied from the video stated above ##

def trace(root):
    """Builds a set of all nodes and edges in a graph"""
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._children:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def visualize(root):
    """Visualizes a directed graph."""
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.name, n.data, n.grad), shape='record')
        if n._operation:
          # if this value is a result of some operation, create an op node for it
          dot.node(name = uid + n._operation, label = n._operation)
          # and connect this node to it
          dot.edge(uid + n._operation, uid)

    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._operation)

    return dot

Now its time to build up a small **Multilayer Perceptron** using `Pulse`. But before we make our hands dirty and build up a MLP we need to construct its minimal unit of layers which consists of Neuron. We can think of this neuron as a Pulse object and n number of neurons make a layer. So lets create that.  

In [4]:
## Neuron implementation

class Neuron:
    """Creates a single neuron."""
    def __init__(self , num_prev_neurons):
        self.weights = []
        for idx in range(num_prev_neurons):   
            self.weights.append(Pulse(data = random.uniform(-1 , 1)))
        
        self.bias = Pulse(data = random.uniform(-1 , 1))
    
    def __call__(self , x):
        return sum((w_i * x_i for w_i , x_i in zip(self.weights , x)) , self.bias).tanh()
    
    def parameters(self):
        return self.weights + [self.bias]

## Layer Implementation

class Layer:
    """Creates a layer of neurons."""
    def __init__(self , num_in_neurons , num_out_neurons):
        self.neurons = [Neuron(num_in_neurons) for _ in range(num_out_neurons)]
    
    def __call__(self , x):
        out =  [neuron(x) for neuron in self.neurons]
        if len(out) == 1:
            return out[0]
        else:
            return out
    def parameters(self):
        return [each_parameter for neuron in self.neurons for each_parameter in neuron.parameters()]
    
class MLP:
    """Creates a multi-layer perceptron."""
    def __init__(self , in_size , out_sizes):
        layer_sizes = [in_size] + out_sizes
        self.layers = [Layer(inp_size , out_size) for idx , (inp_size , out_size) in enumerate(zip(
            layer_sizes , layer_sizes[1:]
        ))]
    
    def __call__(self , x):
        for layer in self.layers:
            x = layer(x)
        
        return x
    
    def parameters(self):
        return [each_parameter for layer in self.layers for each_parameter in layer.parameters()]